<a href="https://colab.research.google.com/github/charlesm93/stanTutorial/blob/main/Nordic_Prob_AI/CourseScript_probAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nording Probabilistic AI summer school: Monte Carlo methods

2024. Instructor: Charles Margossian

This notebook contains R scripts to do the exercises. Throughout, we will use Stan. The scripts below install Stan, as well as some other useful packages.

In [ ]:
rm(list = ls())
gc()

set.seed(1954)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1466891,78.4,2643336,141.2,2643336,141.2
Vcells,2772786,21.2,10146329,77.5,10146317,77.5


In [ ]:
# Set size of output plots (adjust to your preferences)
options(repr.plot.width = 7, repr.plot.height = 3.5, repr.plot.res = 200)

## Install requisite packages

In [ ]:
install_packages <- TRUE
if (install_packages) {
  install.packages("rjson")
  install.packages("posterior")
  install.packages("bayesplot")
  install.packages("outbreaks")
  install.packages("loo")
  install.packages("gridExtra")
}

In [ ]:
if (install_packages) {
  install.packages("cmdstanr", repos = c("https://mc-stan.org/r-packages/", getOption("repos")))
  library(cmdstanr)
  install_cmdstan()
}

## Load packages

In [ ]:
library(rjson)
library(bayesplot)
library(posterior)
library(ggplot2)
library(cmdstanr)
library(parallel)
library(loo)
library(outbreaks)
library(gridExtra)

In [ ]:
mc.cores = detectCores()

Utility function for analyzing the output of `loo` package.

In [ ]:

is_summary <- function (fit, parms, psis_fit, log_ratios) {
  mean <- c()
  var <- c()
  q5 <- c()
  q25 <- c()
  q50 <- c()
  q75 <- c()
  q95 <- c()
  khat <- c()

  for (i in 1:length(parms)) {
    parm_draw <- fit$draws(parms[i])
    mean <- c(mean, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "mean")$value)
    var <- c(var, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "var")$value)
    q5 <- c(q5, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "quantile", probs = 0.05)$value)
    q25 <- c(q25, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "quantile", probs = 0.25)$value)
    q50 <- c(q50, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "quantile", probs = 0.5)$value)
    q75 <- c(q75, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "quantile", probs = 0.75)$value)
    q95 <- c(q95, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "quantile", probs = 0.95)$value)
    khat <- c(khat, E_loo(parm_draw, psis_fit, log_ratios = log_ratios, type = "mean")$pareto_k)
  }

  summary <- data.frame(parms = parms, mean = mean, var = var, q5 = q5, q25 = q25, q50 = q50,
                        q75 = q75, q95 = q95, khat = khat)
  return(summary)
}


## Bayesian Linear Regression



In [ ]:
data <- list(N = 10,
             x = 1:10,
             y = c(12.03037, 13.86786, 16.00795, 18.29419, 20.26679, 22.38664,
                   23.95751, 25.71993, 28.09961, 30.46859))

# define starting distribution
init <- function() {
  list(sigma = rgamma(1, 1),
       beta = rnorm(1, mean = 1, sd = 1))
}

In [ ]:
# transpile (translate Stan to C++ and then compile)
mod <- cmdstan_model("model/linear.stan")

In [ ]:
# run sampler
fit <- mod$sample(data = data, chains = n_chains,
                  init = init,
                  save_warmup = TRUE,
                  parallel_chains = 4,
                  num_warmup = 1000,
                  num_samples = 1000)


In [ ]:
# Examine Stan's default summaries
fit$summary()

In [ ]:
# Construct diagnostic plots
pars <- c("beta", "sigma")
bayesplot::mcmc_trace(fit$draws(inc_warmup = TRUE),
                      n_warmup = 1000, pars = pars)


In [ ]:
bayesplot::mcmc_dens_overlay(fit$draws(), pars = pars)

In [ ]:
# Extract posterior predictive checks
yrep <- as.matrix(
  as_draws_df(fit$draws(variables = c("y_pred"))))
head(yrep)

In [ ]:
# We don't need the chain, iteration and draw ID, so let's remove them.
yrep <- yrep[, -(11:13)]

# Plot the posterior predictions and compare it to the real data.
bayesplot::ppc_ribbon(y = data$y, yrep = yrep, x = data$x,
                      y_draw = "point") +
  theme_bw() +
  ylab("y")


## Disease Transmission model

In [ ]:
theme_set(theme_bw())
ggplot(data = influenza_england_1978_school) +
  geom_point(mapping = aes(x = date, y = in_bed)) +
  labs(y = "Number of students in bed")


In [ ]:
# create a data list to be passed to Stan
cases <- influenza_england_1978_school$in_bed
N <- 763;
n_days <- length(cases)
t <- seq(0, n_days, by = 1)
t0 = 0
t <- t[-1]

#initial conditions
i0 <- 1
s0 <- N - i0
r0 <- 0
y0 = c(S = s0, I = i0, R = r0)

data_sir <- list(n_days = n_days, y0 = y0, t0 = t0, ts = t,
                 N = N, cases = cases)


In [ ]:
# define starting distribution
init <- function() {
  list(beta = abs(rnorm(1, mean = 2, sd = 1)),
       gamma = abs(rnorm(1, mean = 0.4, sd = 0.5)),
       phi_inv = rexp(1, rate = 5))
}

# transpile (translate Stan to C++ and then compile)
mod <- cmdstan_model("model/sir.stan")

n_chains <- 4
fit <- mod$sample(data = data_sir,
                  init = init,
                  save_warmup = TRUE,
                  parallel_chains = 4,
                  num_warmup = 1000,
                  num_samples = 1000)


In [ ]:
pars <- c("gamma", "beta", "phi", "R0")
fit$summary(variables = pars)

bayesplot::mcmc_trace(fit$draws(inc_warmup = TRUE),
                      n_warmup = 1000, pars = pars)
bayesplot::mcmc_dens_overlay(fit$draws(), pars = pars)

# Extract posterior predictive checks
pred_cases <- as.matrix(
  as_draws_df(fit$draws(variables = c("pred_cases"))))[, -(15:17)]

bayesplot::ppc_ribbon(y = data_sir$cases, yrep = pred_cases,
                      x = data_sir$ts, y_draw = "point") +
  theme_bw() +
  ylab("cases") + xlab("days")

In [ ]:
## Run same model with a Poisson likelihood

mod <- cmdstan_model("model/sir_poisson.stan")

fit_poisson <- mod$sample(data = data_sir,
                          init = init,
                          save_warmup = TRUE,
                          parallel_chains = 4,
                          num_warmup = 1000,
                          num_samples = 1000)



In [ ]:
fit_poisson$summary(variables = pars)

pred_cases_poisson <- as.matrix(
  as_draws_df(fit_poisson$draws(variables = c("pred_cases"))))[, -(15:17)]

bayesplot::ppc_ribbon(y = data_sir$cases, yrep = pred_cases_poisson,
                      x = data_sir$ts, y_draw = "point") +
  theme_bw() +
  ylab("cases") + xlab("days")

In [ ]:
# compute PSIS-loo estimate

log_lik_draws <- fit$draws("log_lik")
loo_estimate <- loo(log_lik_draws, r_eff = relative_eff(log_lik_draws))


log_lik_draws_poisson <- fit_poisson$draws("log_lik")
loo_estimate_poisson <-
  loo(log_lik_draws_poisson, r_eff = relative_eff(log_lik_draws_poisson))

print(loo_estimate_poisson)
print(loo_estimate)